In [ ]:
%pip install rdflib

In [ ]:
import google
import os

# prompt: mount drive here to read files from the folder "My Drive > Colab_Notebooks > Formal_Ontology_of_Mathematics"
google.colab.drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Colab_Notebooks/Formal_Ontology_of_Mathematics")
!ls
#Now you can access files in this directory.  For example:
#with open("my_file.txt", "r") as f:
#    file_contents = f.read()
#    print(file_contents)


In [ ]:
import os
import re
import rdflib

import modules.utils as utils
import modules.tbox as tbox
import modules.concepts as concepts

# common IRIs
rdf_type = rdflib.RDF.type
rdfs_label = rdflib.RDFS.label
rdfs_subclassof = rdflib.RDFS.subClassOf
skos_prefLabel = rdflib.SKOS.prefLabel
skos_altLabel = rdflib.SKOS.altLabel
owl_class = rdflib.OWL.Class
owl_individual = rdflib.OWL.NamedIndividual
owl_object_property = rdflib.OWL.ObjectProperty
owl_data_property = rdflib.OWL.DatatypeProperty
owl_annotation_property = rdflib.OWL.AnnotationProperty

concept_class = utils.create_iri("Concept", namespace="https://www.foom.com/core")
definition_refers_to = utils.create_iri("definition refers to", namespace="https://www.foom.com/core")
is_defined_in = utils.create_iri("is defined in", namespace="https://www.foom.com/core")
contains_definition_of = utils.create_iri("contains_definition_of", namespace="https://www.foom.com/core")
is_in = utils.create_iri("is in", namespace="https://www.foom.com/core")
has_definition = utils.create_iri("has definition", namespace="https://www.foom.com/core")
defines = utils.create_iri("defines", namespace="https://www.foom.com/core")

In [ ]:

# initialize graph
kg = rdflib.Graph()

# tbox and foundational triples
ontology_items = {
    ("Concept", owl_class),
    ("is defined in", owl_object_property),
    ("Document", owl_class),
    ("Definition", owl_class),
    ("Postulate", owl_class),
    ("Common notion", owl_class),
    ("Proposition", owl_class)
    ("Statement", owl_class)
    ("Enumeration", owl_class)
    ("Proposition type", owl_class)
    ("contains_definition_of", owl_object_property),
    ("definition refers to", owl_object_property),
    ("is in", owl_object_property),
    ("has definition", owl_object_property),
    ("defines", owl_object_property),
    ("has subject", owl_object_property),
    ("has predicate", owl_object_property),
    ("has object", owl_object_property)
}
triples = {
    (utils.create_iri("Elements Book 1"), rdf_type, owl_individual),
    (utils.create_iri("Elements Book 1"), rdfs_label, rdflib.Literal("Document: Elements Book 1")),
    (utils.create_iri("Elements Book 1"), skos_prefLabel, rdflib.Literal("Elements Book 1")),
    (utils.create_iri("Elements Book 1"), rdf_type, utils.create_iri("Document", namespace="https://www.foom.com/core")),
    (utils.create_iri("Proposition type", namespace="https://www.foom.com/core"), rdfs_subclassof, utils.create_iri("Enumeration", namespace="https://www.foom.com/core")),
}

kg = tbox.add_tbox(kg, ontology_items, triples)

# abox: concepts
input_file_path = "input/ontology_definitions_concepts.txt"
kg = concepts.main_add_definition_concepts(input_file_path, kg)

# output ontology
utils.output_ontology(kg, "output", "euclid_book1.ttl", "turtle")

In [ ]:
def add_postulates(kg: rdflib.Graph,
                   input_file_path: str) -> rdflib.Graph:
    lines = utils.read_text_file(input_file_path)
    postulates = organize_postulates(lines)
    print(postulates)

    # add postulates
    kg = add_postulate_triples(kg, postulates)
    # add links from postulates to concepts and vice versa

    # add relations

    # add operations


    return kg

def add_postulate_triples(kg: rdflib.Graph,
                          postulates: dict) -> rdflib.Graph:
    for postulate in postulates:
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/euclid")
        postulate_label = rdflib.Literal(postulate.replace("_", " "))
        kg.add((postulate_iri, rdfs_label, rdflib.Literal(postulate_label)))
        kg.add((postulate_iri, rdf_type, owl_individual))
        kg.add((postulate_iri, rdf_type, utils.create_iri("Postulate", namespace="https://www.foom.com/core")))
        kg.add((postulate_iri, rdfs_label, postulate_label))
        kg.add((postulate_iri, skos_prefLabel, postulate_label))
    return kg


def organize_postulates(lines: list) -> dict:
    postulates = {
        line.split()[0]: {
            "concepts": [],
            "relations": [],
            "operations": []
        }
        for line in lines
    }
    for line in lines:
        items = [item.replace(":", "").replace(",", "").strip() for item in line.split()]
        for item in items[2:]:
            postulates[items[0]][items[1]].append(item)
    return postulates

input_file_path = "input/ontology_postulates.txt"
add_postulates(kg, input_file_path)